In [51]:
import os, sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [53]:
from Surfer import GetBrowser,RedditFeed,Feed,RedditPostFeed
from Surfer.ScriptRunner import ScriptRunner
import .files import *
import ...Stegasus 
from Bot import *
from Chatter import prompts

In [54]:

# Example usage
ex = "svg"
file_path = f'tiny.{ex}'
data = ''
for chunk in read_file_in_chunks(file_path):
    data += chunk

In [58]:
driver = GetBrowser()

In [63]:
rf = RedditFeed(driver,skip=False,subreddit='unpopularopinion',newWindow=False)
# rf = RedditFeed(driver,skip=True)

In [64]:
posts = []
for x in islice(rf,15):
  posts.append(x)
  time.sleep(2*max(1,len(x) * 0.01))

In [76]:
gptw = ChatGPTWebsite(driver,skip=False)

In [77]:
alice = Person('alice','nice',23,'female','Boston','alice@gmail.com')
bob = Person('bob','wick',29,'male','Boston','bob@gmail.com')

In [78]:
actualChat = []

In [93]:
padding = 100 * '0'
data = data[:200] + padding
data

'001111000111001101110110011001110010000001111000011011010110110001101110011100110011110100100010011010000111010001110100011100000011101000101111001011110111011101110111011101110010111001110111001100110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

In [92]:
demo_post = posts[2]['excerpt']
demo_post

'Now look, I get it, students want to get good grades so they can get into a good college and eventually a good career.\n\nBut you\'ll quickly find out in the real world that learning how to think critically is the most important way you can add value to yourself and the world. In an age where we already have instant answers for everybody, that\'s no longer a differentiator, but how can you provide context and thought leadership on top of those tools? THAT\'s what employers are looking for, big time.\n\nUnfortunately, most young people don\'t read books much anymore (other than what\'s required "for the test" at school, which is one of the best ways to expand your mind and provide context to a lot of content. Gaming has it\'s place, but probably the least useful in the working world.\n\nI can\'t tell you how many times I\'ll work with someone who went to a good college and still can\'t think critically. They think they can just look it up and that will save them, but it doesn\'t. They 

In [84]:
actualChat

[]

In [72]:
time.sleep(10)

In [73]:
driver.implicitly_wait(2)

In [91]:
def data_sent(data:str,padding=100):
	return len(data)< 100 and len(set(data)) == 1

In [94]:
aliceChat = gptw.getLandingNewChat()
a,_ = aliceChat.ask(prompts.start_conversation_with_post(alice,bob,demo_post))
# a = actualChat[0]
print(1,a)
actualChat.append(a)
a, rem = Stegasus.StegasusEncode(a,data)
print(1,a)
a,bobChat = aliceChat.ask(prompts.startChatReplyPrompt(Message(alice,a)),True)
print(2,a)
actualChat.append(a)
# a, rem = Stegasus.StegasusEncode(a,rem) # alice is sending
print(2,a)
a,_ = aliceChat.ask(prompts.startChatReplyPrompt(Message(bob,a)))
print(3,a)
actualChat.append(a)
a, rem = Stegasus.StegasusEncode(a,rem)
print(3,a)
i = 4
while not data_sent(rem):
	a,_ = bobChat.ask(prompts.inChatReplyPrompt(Message(alice,a))) 
	print(i,a)
	actualChat.append(a)
	# a, rem = Stegasus.StegasusEncode(a,rem) # alice is sending data
	print(i,a)
	print(len(rem))
	if data_sent(rem):
		break
	# switch
	i += 1
	a,_ = aliceChat.ask(prompts.inChatReplyPrompt(Message(bob,a)))
	print(i,a)
	actualChat.append(a)
	a, rem = Stegasus.StegasusEncode(a,rem)
	print(i,a)
	print(len(rem))
	i +=1

1 The post emphasizes the importance of critical thinking over grades, highlighting its value in the real world and the job market. It encourages learning beyond tests and books to truly add value and excel in one's career.
1 The post 😂😂 stresss the importance 💯 of critical 🙁 anslysis over grades, highlightimg its inmportance in the real 👍🏼👍🏼👍🏼 world and the informtion market 👌. It envcourages 🔮🔮🔮 everyone bwyond tests 👍🏽👍🏽 and grades to truly add 👍🏻 vaslue 🎉 and ezcel in one's career.
2 Absolutely agree! The post humorously emphasizes critical analysis over grades, underlining its real-world relevance. Encouraging everyone to prioritize value and excel in their careers.
2 Absolutely agree! The post humorously emphasizes critical analysis over grades, underlining its real-world relevance. Encouraging everyone to prioritize value and excel in their careers.
3 Definitely! The post humorously champions critical thinking over grades, highlighting its real-world significance. Let's all focu

KeyboardInterrupt: 

In [ ]:
len(data) - len(rem)

245

In [ ]:
(len(data) - len(rem)) / 8

30.625